In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
%set_env CUDA_LAUNCH_BLOCKING=1

env: CUDA_LAUNCH_BLOCKING=1


In [4]:
import sys
import collections
import cv2
import os
import math
import random
import pickle
import copy
import numpy as np
import pandas as pd
sys.path.append("C:\\Users\\user\\Documents\\Workspace\\MeronymNet-PyTorch\\src")

In [5]:
import torch
import torch.optim as optim
from torch_geometric.data import Data
from torch_geometric.utils import dense_to_sparse
from torch_geometric.data import DataLoader
import torch.utils.data as data_utils
from torch.utils.tensorboard import SummaryWriter

In [6]:
from losses import BoxVAE_losses as loss
from evaluation import metrics
from utils import plot_utils
from utils import data_utils as data_loading
from components.DenseAutoencoder import DenseAutoencoder
from components.DenseAutoencoder import Decoder
from components.TwoStageAutoEncoder import TwoStageAutoEncoder

In [7]:
import matplotlib.pyplot as plt

In [8]:
# from mask_generation import masked_sketch

In [15]:
latent_dims = 64
batch_size = 128
num_nodes = 16
bbx_size = 4
num_classes = 7
label_shape = 1
nb_epochs = 400
klw = loss.frange_cycle_linear(nb_epochs)
learning_rate = 0.000055
hidden1 = 32
hidden2 = 16
hidden3 = 128
dense_hidden1=16
dense_hidden2=32
adaptive_margin = True
fine_tune_box = False
output_log = False
area_encoding = False
run_prefix = "two_stage_no_obj_mse"
variational=False

In [14]:
batch_train_loader, batch_val_loader = data_loading.load_data(obj_data_postfix = '_obj_boundary',
                                                              part_data_post_fix = '_scaled',
                                                              file_postfix = '_combined',
                                                              seed=345,
                                                              batch_size=batch_size)

C:\Users\user\miniconda3\lib\site-packages\torch_geometric\deprecation.py:13: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


In [17]:
import gc
gc.collect()

63

In [16]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [18]:
reconstruction_loss_arr = []
kl_loss_arr = []
bbox_loss_arr = []
adj_loss_arr = []
node_loss_arr = []

reconstruction_loss_val_arr = []
kl_loss_val_arr = []
bbox_loss_val_arr = []
adj_loss_val_arr = []
node_loss_val_arr = []

vae = TwoStageAutoEncoder(latent_dims,
                          num_nodes,
                          bbx_size,
                          num_classes,
                          label_shape,
                          hidden1,
                          hidden2,
                          hidden3,
                          dense_hidden1,
                          dense_hidden2,
                          adaptive_margin,
                          output_log,
                          area_encoding
                        )
vae.to(device)
optimizer = optim.Adam(vae.parameters(), lr=learning_rate)
#scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=[200,400], gamma=0.75)
model_path = ('D:/meronym_data/model/'+run_prefix+'/Obj-Box-'
                        +str(learning_rate)
                        +'-batch-'+str(batch_size)
                        +'-h1-'+str(hidden1)
                        +'-h2-'+str(hidden2)
                        +'-h3-'+str(hidden3)+'-test')
summary_path = ('D:/meronym_data/runs/'+run_prefix+'/Obj-Box-'
                        +str(learning_rate)
                        +'-batch-'+str(batch_size)
                        +'-h1-'+str(hidden1)
                        +'-h2-'+str(hidden2)
                        +'-h3-'+str(hidden3)+'-test')
if not os.path.exists(model_path):
    os.makedirs(model_path)
writer = SummaryWriter(summary_path)
icoef = 0

for epoch in range(nb_epochs):
    
    batch_loss = torch.tensor([0.0])
    batch_kl_loss = torch.tensor([0.0])
    batch_bbox_loss = torch.tensor([0.0])
    batch_obj_bbox_loss = torch.tensor([0.0])
    batch_node_loss = torch.tensor([0.0])
    images = []
    
    vae.train()
    i=0
    for train_data in batch_train_loader:
        
        node_data_true = train_data.x
        label_true = node_data_true[:,:1]
        y_true = train_data.y
        class_true = y_true[:, :num_classes]
        X_obj_true = y_true[:, num_classes:]
        adj_true = train_data.edge_index
        batch = train_data.batch
#         node_data_true  = torch.reshape(node_data_true, (int(batch.shape[0]/num_nodes), num_nodes, node_data_true.shape[-1]))
        
        class_true  = torch.flatten(class_true)
        

        for param in vae.parameters():
            param.grad=None
        
        output = vae(adj_true, node_data_true, X_obj_true, label_true , class_true)
        node_data_pred, X_obj_pred, label_pred, z_mean, z_logvar, margin = output
        obj_bbox_loss = loss.obj_bbox_loss(pred_box=X_obj_pred, true_box=X_obj_true, has_mse=False)
        kl_loss = loss.kl_loss(z_mean, z_logvar)
        bbox_loss = loss.bbox_loss(pred_box=node_data_pred, true_box=node_data_true[:,1:], margin=margin)
        node_loss = loss.node_loss(label_pred,label_true)
        
        kl_weight = klw[icoef]
        
        if kl_weight>0:
            reconstruction_loss = kl_loss*kl_weight + (obj_bbox_loss + bbox_loss + node_loss)*num_nodes
        else:
            reconstruction_loss = (obj_bbox_loss + bbox_loss + node_loss)*num_nodes
            
        reconstruction_loss.backward()
        
        optimizer.step()
        
        i+=1
      
        batch_loss += reconstruction_loss
        batch_kl_loss += kl_loss
        batch_bbox_loss += bbox_loss
        batch_obj_bbox_loss += obj_bbox_loss
        batch_node_loss += node_loss
    
        if i%200==0:
            print(i)
            global_step = epoch*len(batch_train_loader)+i
            
            writer.add_scalar("Loss/train/reconstruction_loss", batch_loss.item()/(i+1), global_step)
            writer.add_scalar("Loss/train/kl_loss", batch_kl_loss.item()/(i+1), global_step)
            writer.add_scalar("Loss/train/bbox_loss", batch_bbox_loss.item()/(i+1), global_step)
            writer.add_scalar("Loss/train/obj_bbox_loss", batch_obj_bbox_loss.item()/(i+1), global_step)
            writer.add_scalar("Loss/train/node_loss", batch_node_loss.item()/(i+1), global_step)
            
            
    #scheduler.step()
    global_step = epoch*len(batch_train_loader)+i
    image_shape = [num_nodes, bbx_size]

    image = plot_utils.plot_bbx(np.reshape((node_data_true[:num_nodes,1:5]*label_true[:num_nodes]).detach().to("cpu").numpy(),
                                image_shape)).astype(float)/255
    writer.add_image('train/images/input', image, global_step, dataformats='HWC')
    image = plot_utils.plot_bbx((node_data_pred[0]*label_true[:num_nodes]).detach().to("cpu").numpy()).astype(float)/255
    writer.add_image('train/images/generated', image, global_step, dataformats='HWC')
    
    reconstruction_loss_arr.append(batch_loss.detach().item()/(i+1))
    kl_loss_arr.append(batch_kl_loss.detach().item()/(i+1))
    bbox_loss_arr.append(batch_bbox_loss.detach().item()/(i+1))
    node_loss_arr.append(batch_node_loss.detach().item()/(i+1))
    
    print('[%d, %5d] loss: %.3f' %
              (epoch + 1, i + 1, batch_loss/(i+1) ))
    
    
    batch_loss = torch.tensor([0.0])
    batch_kl_loss = torch.tensor([0.0])
    batch_bbox_loss = torch.tensor([0.0])
    batch_obj_bbox_loss = torch.tensor([0.0])
    batch_node_loss = torch.tensor([0.0])
    images = []
    vae.eval()
    for i, val_data in enumerate(batch_val_loader, 0):
        
        node_data_true = val_data.x
        label_true = node_data_true[:,:1]
        y_true = val_data.y
        class_true = torch.flatten(y_true[:, :num_classes])
        X_obj_true = y_true[:, num_classes:]
        adj_true = val_data.edge_index
        batch = val_data.batch
        
        class_true  = torch.flatten(class_true)
        
        output = vae(adj_true, node_data_true, X_obj_true, label_true , class_true)
        node_data_pred, X_obj_pred, label_pred, z_mean, z_logvar, margin = output
        obj_bbox_loss = loss.obj_bbox_loss(pred_box=X_obj_pred, true_box=X_obj_true, has_mse=False)
        kl_loss = loss.kl_loss(z_mean, z_logvar)
        bbox_loss = loss.bbox_loss(pred_box=node_data_pred, true_box=node_data_true[:,1:], margin=margin)
        node_loss = loss.node_loss(label_pred,label_true)
        
        if kl_weight>0:
            reconstruction_loss = kl_loss*kl_weight + (obj_bbox_loss + bbox_loss + node_loss)*num_nodes
        else:
            reconstruction_loss = (obj_bbox_loss + bbox_loss + node_loss)*num_nodes
            
        batch_loss += reconstruction_loss
        batch_kl_loss += kl_loss
        batch_bbox_loss += bbox_loss
        batch_obj_bbox_loss += obj_bbox_loss
        batch_node_loss += node_loss
    
    image = plot_utils.plot_bbx(np.reshape((node_data_true[:num_nodes,1:5]*label_true[:num_nodes]).detach().to("cpu").numpy(),
                                image_shape)).astype(float)/255
    writer.add_image('val/images/input', image, global_step, dataformats='HWC')
    image = plot_utils.plot_bbx((node_data_pred[0]*label_true[:num_nodes]).detach().to("cpu").numpy()).astype(float)/255
    writer.add_image('val/images/generated', image, global_step, dataformats='HWC')

    reconstruction_loss_val_arr.append(batch_loss.detach().item()/(i+1))
    kl_loss_val_arr.append(batch_kl_loss.detach().item()/(i+1))
    bbox_loss_val_arr.append(batch_bbox_loss.detach().item()/(i+1))
    node_loss_val_arr.append(batch_node_loss.detach().item()/(i+1))
    
    writer.add_scalar("Loss/val/reconstruction_loss", batch_loss.detach()/(i+1), global_step)
    writer.add_scalar("Loss/val/kl_loss", batch_kl_loss.item()/(i+1), global_step)
    writer.add_scalar("Loss/val/bbox_loss", batch_bbox_loss.item()/(i+1), global_step)
    writer.add_scalar("Loss/val/obj_bbox_loss", batch_obj_bbox_loss.item()/(i+1), global_step)
    writer.add_scalar("Loss/val/node_loss", batch_node_loss.item()/(i+1), global_step)
       
    if epoch%50 == 0:
        torch.save(vae.state_dict(), model_path + '/model_weights.pth')
        
#     if kl_loss_arr[-1]>0.5 and abs(bbox_loss_arr[-1] - bbox_loss_val_arr[-1]) < 0.02 and bbox_loss_arr[-1]<1.3 and epoch>60:
#         icoef = icoef + 1  

torch.save(vae.state_dict(),model_path + '/model_weights.pth')

for i in range(min(100,int(len(node_data_true)/num_nodes))):    
    image = plot_utils.plot_bbx(np.reshape((node_data_true[num_nodes*i:num_nodes*(i+1),1:5]).detach().to("cpu").numpy(),
                                    image_shape)).astype(float)/255
    writer.add_image('result/images/'+str(i)+'-input', image, global_step, dataformats='HWC')
    image = plot_utils.plot_bbx((node_data_pred[i]*label_true[num_nodes*i:num_nodes*(i+1)]).detach().to("cpu").numpy()).astype(float)/255
    writer.add_image('result/images/'+str(i)+'-generated', image, global_step, dataformats='HWC')
    
writer.flush()
writer.close()
print('Finished Training')

200
[1,   399] loss: 138.623
200
[2,   399] loss: 99.718
200
[3,   399] loss: 94.849
200
[4,   399] loss: 93.288
200
[5,   399] loss: 92.558
200
[6,   399] loss: 91.717
200
[7,   399] loss: 91.207
200
[8,   399] loss: 90.857
200
[9,   399] loss: 90.433
200
[10,   399] loss: 89.677
200
[11,   399] loss: 87.686
200
[12,   399] loss: 83.425
200
[13,   399] loss: 79.220
200
[14,   399] loss: 75.605
200
[15,   399] loss: 71.084
200
[16,   399] loss: 66.364
200
[17,   399] loss: 62.751
200
[18,   399] loss: 59.831
200
[19,   399] loss: 57.132
200
[20,   399] loss: 54.641
200
[21,   399] loss: 52.311
200
[22,   399] loss: 49.810
200
[23,   399] loss: 47.575
200
[24,   399] loss: 46.042
200
[25,   399] loss: 44.834
200
[26,   399] loss: 43.827
200
[27,   399] loss: 42.965
200
[28,   399] loss: 42.198
200
[29,   399] loss: 41.493
200
[30,   399] loss: 40.826
200
[31,   399] loss: 40.197
200
[32,   399] loss: 39.594
200
[33,   399] loss: 39.017
200
[34,   399] loss: 38.458
200
[35,   399] loss: 

200
[278,   399] loss: 10.426
200
[279,   399] loss: 10.400
200
[280,   399] loss: 10.370
200
[281,   399] loss: 10.342
200
[282,   399] loss: 10.317
200
[283,   399] loss: 10.289
200
[284,   399] loss: 10.263
200
[285,   399] loss: 10.235
200
[286,   399] loss: 10.208
200
[287,   399] loss: 10.180
200
[288,   399] loss: 10.154
200
[289,   399] loss: 10.128
200
[290,   399] loss: 10.100
200
[291,   399] loss: 10.073
200
[292,   399] loss: 10.047
200
[293,   399] loss: 10.022
200
[294,   399] loss: 9.993
200
[295,   399] loss: 9.968
200
[296,   399] loss: 9.942
200
[297,   399] loss: 9.915
200
[298,   399] loss: 9.890
200
[299,   399] loss: 9.862
200
[300,   399] loss: 9.838
200
[301,   399] loss: 9.813
200
[302,   399] loss: 9.786
200
[303,   399] loss: 9.760
200
[304,   399] loss: 9.734
200
[305,   399] loss: 9.709
200
[306,   399] loss: 9.685
200
[307,   399] loss: 9.658
200
[308,   399] loss: 9.636
200
[309,   399] loss: 9.608
200
[310,   399] loss: 9.584
200
[311,   399] loss: 9.56

In [19]:
#testing loop
model_path = ('D:/meronym_data/model/'+run_prefix+'/Obj-Box-'
                        +str(learning_rate)
                        +'-batch-'+str(batch_size)
                        +'-h1-'+str(hidden1)
                        +'-h2-'+str(hidden2)
                        +'-h3-'+str(hidden3)+'-test')
summary_path = ('D:/meronym_data/runs/'+run_prefix+'/Obj-Box-'
                        +str(learning_rate)
                        +'-batch-'+str(batch_size)
                        +'-h1-'+str(hidden1)
                        +'-h2-'+str(hidden2)
                        +'-h3-'+str(hidden3)+'-test')


write_tensorboard = False
if write_tensorboard:
    writer = SummaryWriter(summary_path)

vae = TwoStageAutoEncoder(latent_dims,
                          num_nodes,
                          bbx_size,
                          num_classes,
                          label_shape,
                          hidden1,
                          hidden2,
                          hidden3,
                          dense_hidden1,
                          dense_hidden2,
                          adaptive_margin,
                          output_log,
                          area_encoding
                        )
vae.load_state_dict(torch.load(model_path+ '/model_weights.pth'))

# decoder = vae.decoder
image_shape = [num_nodes, bbx_size]
global_step = 250000
class_dict = {0:'cow', 1:'sheep', 2:'bird', 3:'person', 4:'cat', 5:'dog', 6:'horse'}
res_dfs = []
for i, val_data in enumerate(batch_val_loader, 0):
    
    val_data.cuda()
    node_data_true = val_data.x
    label_true = node_data_true[:,:1]
    y_true = val_data.y
    class_true = y_true[:, :num_classes]
    X_obj_true = y_true[:, num_classes:]
    adj_true = val_data.edge_index
    class_true  = torch.flatten(class_true)
    
    output = vae(adj_true, node_data_true, X_obj_true, label_true , class_true)
    node_data_pred_test, X_obj_pred_test, label_pred, z_mean, z_logvar, margin = output
    
    res_dfs.append(metrics.get_metrics(node_data_true, X_obj_true, node_data_pred_test,
                               X_obj_pred_test,
                               label_true,
                               class_true,
                               num_nodes,
                               num_classes))
    
    if write_tensorboard:
        
        for j in range(int(len(node_data_true)/num_nodes)):
            
            image = plot_bbx(node_data_true[j].detach().to("cpu").numpy().astype(float))/255
            pred_image = plot_bbx(node_data_pred_test[j].detach().to("cpu").numpy()).astype(float)/255

            writer.add_image('test_result/images/'+str(j)+'-input/', image, global_step, dataformats='HWC')  
            writer.add_image('test_result/images/'+str(j)+'-generated/', pred_image, global_step, dataformats='HWC')
            

result = pd.concat(res_dfs)
result['obj_class'] = np.where(result['obj_class'].isna(), 0, result['obj_class'])
result['obj_class'] = result['obj_class'].astype('int')
result['obj_class'].replace(class_dict, inplace=True)
result.where(result['part_labels']!=0, np.NaN, inplace=True)
result['part_labels'] = np.where(result['part_labels'].isna(), 0, result['part_labels'])
result['part_labels'] = result['part_labels'].astype('int')
result['id'] = np.repeat(np.array(list(range(int(len(result)/num_nodes)))), 16)

if write_tensorboard:
    writer.flush()
    writer.close()

bird_labels = {'head':1 , 'torso':2, 'neck':3, 'lwing':4, 'rwing':5, 'lleg':6, 'lfoot':7, 'rleg':8, 'rfoot':9, 'tail':10}
cat_labels = {'head':1, 'torso':2, 'neck':3, 'lfleg':4, 'lfpa':5, 'rfleg':6, 'rfpa':7, 'lbleg':8, 'lbpa':9, 'rbleg':10, 'rbpa':11, 'tail':12}
cow_labels = {'head':1,'lhorn':2, 'rhorn':3, 'torso':4, 'neck':5, 'lfuleg':6, 'lflleg':7, 'rfuleg':8, 'rflleg':9, 'lbuleg':10, 'lblleg':11, 'rbuleg':12, 'rblleg':13, 'tail':14}
dog_labels = {'head':1,'torso':2, 'neck':3, 'lfleg':4, 'lfpa':5, 'rfleg':6, 'rfpa':7, 'lbleg':8, 'lbpa':9, 'rbleg':10, 'rbpa':11, 'tail':12, 'muzzle':13}
horse_labels = {'head':1,'lfho':2, 'rfho':3, 'torso':4, 'neck':5, 'lfuleg':6, 'lflleg':7, 'rfuleg':8, 'rflleg':9, 'lbuleg':10, 'lblleg':11, 'rbuleg':12, 'rblleg':13, 'tail':14, 'lbho':15, 'rbho':16}
person_labels = {'head':1, 'torso':2, 'neck': 3, 'llarm': 4, 'luarm': 5, 'lhand': 6, 'rlarm':7, 'ruarm':8, 'rhand': 9, 'llleg': 10, 'luleg':11, 'lfoot':12, 'rlleg':13, 'ruleg':14, 'rfoot':15}
sheep_labels = cow_labels
part_labels_combined_parts = {'bird': bird_labels, 'cat': cat_labels, 'cow': cow_labels, 'dog': dog_labels, 'sheep': sheep_labels, 'horse':horse_labels,'person':person_labels}

for k, v in part_labels_combined_parts.items():
    new_map = {}
    for pk, pv in v.items():
        new_map[pv]=pk
    part_labels_combined_parts[k] = new_map
    
for k, v in part_labels_combined_parts.items():
    result.loc[result['obj_class']==k, ['part_labels']] = result.loc[result['obj_class']==k,['part_labels']].replace(v).copy()

result.to_csv(model_path+'/raw_metrics.csv')
res_obj_level = result.groupby(['obj_class', 'id']).mean().reset_index()
res_obj_level.groupby(['obj_class']).mean().reset_index()[['obj_class', 'IOU', 'MSE']].to_csv(model_path+'/obj_level_metrics.csv')
result.groupby(['obj_class', 'part_labels']).mean().reset_index()[['obj_class', 'part_labels',  'IOU', 'MSE']].to_csv(model_path+'/part_level_metrics.csv')

C:\Users\user\Documents\Workspace\MeronymNet-PyTorch\src\evaluation\metrics.py:21: RuntimeWarning: invalid value encountered in true_divide
  iouk = interArea / unionArea


In [20]:
model_path = ('D:/meronym_data/model/'+run_prefix+'/Obj-Box-'
                        +str(learning_rate)
                        +'-batch-'+str(batch_size)
                        +'-h1-'+str(hidden1)
                        +'-h2-'+str(hidden2)
                        +'-h3-'+str(hidden3)+'-test')
summary_path = ('D:/meronym_data/runs/'+run_prefix+'/Obj-Box-'
                        +str(learning_rate)
                        +'-batch-'+str(batch_size)
                        +'-h1-'+str(hidden1)
                        +'-h2-'+str(hidden2)
                        +'-h3-'+str(hidden3)+'-test')
result = pd.read_csv(model_path+'/raw_metrics.csv')
result.groupby(['obj_class', 'part_labels']).mean().reset_index()[['obj_class', 'part_labels',  'IOU', 'MSE']]

,obj_class,part_labels,IOU,MSE
0,bird,head,0.545547,0.000540
1,bird,lfoot,0.225693,0.000724
2,bird,lleg,0.331794,0.000528
3,bird,lwing,0.373925,0.001874
4,bird,neck,0.458762,0.000507
...,...,...,...,...
81,sheep,lhorn,0.217813,0.000935
82,sheep,neck,0.629173,0.000303
83,sheep,rhorn,0.224045,0.001047
84,sheep,tail,0.332245,0.000399


In [21]:
res_obj_level = result.groupby(['obj_class', 'id']).mean().reset_index()
res_obj_level.groupby(['obj_class']).mean().reset_index()[['obj_class', 'IOU', 'MSE']]

,obj_class,IOU,MSE
0,bird,0.510671,0.000689
1,cat,0.565609,0.001938
2,cow,0.520644,0.000672
3,dog,0.457582,0.001199
4,horse,0.401527,0.001073
5,person,0.474264,0.001235
6,sheep,0.638984,0.000381


In [22]:
res_obj_level[['IOU', 'MSE']].mean()

IOU    0.503452
MSE    0.001117
dtype: float64

In [23]:
result.groupby(['obj_class', 'part_labels']).mean().reset_index()[['obj_class', 'part_labels',  'IOU', 'MSE']]

,obj_class,part_labels,IOU,MSE
0,bird,head,0.545547,0.000540
1,bird,lfoot,0.225693,0.000724
2,bird,lleg,0.331794,0.000528
3,bird,lwing,0.373925,0.001874
4,bird,neck,0.458762,0.000507
...,...,...,...,...
81,sheep,lhorn,0.217813,0.000935
82,sheep,neck,0.629173,0.000303
83,sheep,rhorn,0.224045,0.001047
84,sheep,tail,0.332245,0.000399


In [ ]:
outfile = 'D:/meronym_data/generate_boxes.npy'
with open(outfile, 'wb') as pickle_file:
    pred_boxes = np.concatenate(pred_boxes)
    pickle.dump(pred_boxes, pickle_file)
outfile = 'D:/meronym_data/generate_boxesobj_class.npy'
with open(outfile, 'wb') as pickle_file:
    pickle.dump(classes,pickle_file)